In [5]:
!pip install mpld3
!pip install --upgrade pip
!pip install psycopg2-binary
!pip install pystan==2.19.1.1 prophet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at 

In [6]:
#Import Python Packages
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
from datetime import datetime, timedelta
import time
import psycopg2
import boto3
from prophet import Prophet

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False
import mpld3
mpld3.enable_notebook()

In [7]:
# Put the ARN of your AWS Secrets Manager secret for your redshift cluster here:
secret_name="arn:aws:secretsmanager:us-west-2:451614664738:secret:ForecastingSecret-JRCdLL"

# This will get the secret from AWS Secrets Manager. 
import boto3
import json

session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager'
)

get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

if 'SecretString' in get_secret_value_response:
    connection_info = json.loads(get_secret_value_response['SecretString'])
else:
    print("ERROR: no secret data found")

In [8]:
#Assign a variable that contains a string of your credentials
credentials=psycopg2.connect(
    dbname   = 'exelredshift', 
    host     = connection_info["host"], 
    port     = connection_info["port"], 
    user     = connection_info["username"], 
    password = connection_info["password"]
)

In [10]:
# read in your SQL query results using pandas
df = pd.read_sql("""
            select * from edm.bl_cabo_rcc_hcc_nps
            """, con = credentials)


In [7]:
selection_rcc_comm = df[['week_ending_dt','projected_sp_rcc_sales','projected_sd_rcc_sales']]
selection_hcc_comm = df[['week_ending_dt','projected_sp_hcc_sales','projected_sd_hcc_sales']]
selection_dtc_comm = df[['week_ending_dt','projected_sp_thy_sales','projected_sd_thy_sales']]
selection_ease = df[['week_ending_dt','projected_ease_rcc_sales','projected_ease_hcc_sales','projected_ease_thy_sales']]

In [8]:

sorted_data_rcc_comm= selection_rcc_comm.sort_values(by='week_ending_dt')
sorted_data_hcc_comm= selection_hcc_comm.sort_values(by='week_ending_dt')
sorted_data_dtc_comm= selection_dtc_comm.sort_values(by='week_ending_dt')
sorted_data_ease = selection_ease.sort_values(by='week_ending_dt')


In [9]:
rcc_agg_df= sorted_data_rcc_comm.groupby(["week_ending_dt"]).sum()
hcc_agg_df= sorted_data_hcc_comm.groupby(["week_ending_dt"]).sum()
dtc_agg_df= sorted_data_dtc_comm.groupby(["week_ending_dt"]).sum()
ease_agg_df= sorted_data_ease.groupby(["week_ending_dt"]).sum()


In [10]:
rcc_agg_sorted = rcc_agg_df.sort_values(by='week_ending_dt')
hcc_agg_sorted = hcc_agg_df.sort_values(by='week_ending_dt')
dtc_agg_sorted = dtc_agg_df.sort_values(by='week_ending_dt')
ease_agg_sorted = ease_agg_df.sort_values(by='week_ending_dt')

In [11]:
rcc_dfn= rcc_agg_sorted.reset_index()
hcc_dfn= hcc_agg_sorted.reset_index()
dtc_dfn= dtc_agg_sorted.reset_index()
ease_dfn= ease_agg_sorted.reset_index()

In [12]:

rcc_dfn['y'] = rcc_dfn['projected_sp_rcc_sales']+ rcc_dfn['projected_sd_rcc_sales']
hcc_dfn['y'] = hcc_dfn['projected_sp_hcc_sales']+ hcc_dfn['projected_sd_hcc_sales']
dtc_dfn['y'] = dtc_dfn['projected_sp_thy_sales']+ dtc_dfn['projected_sd_thy_sales']
ease_dfn['y'] = ease_dfn['projected_ease_rcc_sales']+ ease_dfn['projected_ease_hcc_sales']+ ease_dfn['projected_ease_thy_sales']



In [13]:

rcc_new_dfn= rcc_dfn[['week_ending_dt', 'y']]
hcc_new_dfn= hcc_dfn[['week_ending_dt', 'y']]
dtc_new_dfn= dtc_dfn[['week_ending_dt', 'y']]
ease_new_dfn= ease_dfn[['week_ending_dt', 'y']]


In [14]:
#Convert to datetime format
rcc_new_dfn['ds'] = pd.to_datetime(rcc_new_dfn['week_ending_dt'])
hcc_new_dfn['ds'] = pd.to_datetime(hcc_new_dfn['week_ending_dt'])
dtc_new_dfn['ds'] = pd.to_datetime(dtc_new_dfn['week_ending_dt'])
ease_new_dfn['ds'] = pd.to_datetime(ease_new_dfn['week_ending_dt'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [15]:
rcc_sorted_dfn= rcc_new_dfn.sort_values(by='week_ending_dt')
rcc_sorted_dfn.reset_index()

hcc_sorted_dfn= hcc_new_dfn.sort_values(by='week_ending_dt')
hcc_sorted_dfn.reset_index()

dtc_sorted_dfn= dtc_new_dfn.sort_values(by='week_ending_dt')
dtc_sorted_dfn.reset_index()

ease_sorted_dfn= ease_new_dfn.sort_values(by='week_ending_dt')
ease_sorted_dfn.reset_index()

,index,week_ending_dt,y,ds
0,0,2018-01-05,15.50,2018-01-05
1,1,2018-01-12,41.50,2018-01-12
2,2,2018-01-19,37.97,2018-01-19
3,3,2018-01-26,38.96,2018-01-26
4,4,2018-02-02,69.50,2018-02-02
...,...,...,...,...
285,285,2023-06-23,155.50,2023-06-23
286,286,2023-06-30,195.50,2023-06-30
287,287,2023-07-07,149.00,2023-07-07
288,288,2023-07-14,160.50,2023-07-14


In [16]:
rcc_master_dataframe_comm= rcc_sorted_dfn[['ds','y']]
hcc_master_dataframe_comm= hcc_sorted_dfn[['ds','y']]
dtc_master_dataframe_comm= dtc_sorted_dfn[['ds','y']]
ease_master_dataframe_comm= ease_sorted_dfn[['ds','y']]

In [17]:
rcc_master_dataframe_comm['y'] = rcc_master_dataframe_comm['y'].apply(np.ceil)
hcc_master_dataframe_comm['y'] = hcc_master_dataframe_comm['y'].apply(np.ceil)
dtc_master_dataframe_comm['y'] = dtc_master_dataframe_comm['y'].apply(np.ceil)
ease_master_dataframe_comm['y'] = ease_master_dataframe_comm['y'].apply(np.ceil)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [18]:
rcc_master_dataframe_comm = rcc_master_dataframe_comm.astype({"y": int})
hcc_master_dataframe_comm = hcc_master_dataframe_comm.astype({"y": int})
dtc_master_dataframe_comm = dtc_master_dataframe_comm.astype({"y": int})
ease_master_dataframe_comm = ease_master_dataframe_comm.astype({"y": int})

In [19]:
rcc_master_dataframe_cabo = rcc_master_dataframe_comm[['ds','y']]
hcc_master_dataframe_cabo = hcc_master_dataframe_comm[['ds','y']]
dtc_master_dataframe_cabo = dtc_master_dataframe_comm[['ds','y']]
ease_master_dataframe_cabo = ease_master_dataframe_comm[['ds','y']]

In [20]:
rcc_master_dataframe_cabo['date']=pd.to_datetime(rcc_master_dataframe_cabo['ds'])
hcc_master_dataframe_cabo['date']=pd.to_datetime(hcc_master_dataframe_cabo['ds'])
dtc_master_dataframe_cabo['date']=pd.to_datetime(dtc_master_dataframe_cabo['ds'])
ease_master_dataframe_cabo['date']=pd.to_datetime(ease_master_dataframe_cabo['ds'])

In [21]:
rcc_new_df1 = rcc_master_dataframe_cabo[rcc_master_dataframe_cabo['ds'] > '2021-01-01']
hcc_new_df1 = hcc_master_dataframe_cabo[hcc_master_dataframe_cabo['ds'] > '2021-01-01']
dtc_new_df1 = dtc_master_dataframe_cabo[dtc_master_dataframe_cabo['ds'] > '2021-01-01']
ease_new_df1 = ease_master_dataframe_cabo[ease_master_dataframe_cabo['ds'] > '2021-01-01']

In [22]:
#Set date column as the index
rcc_new_df =rcc_new_df1.set_index('date')
hcc_new_df =hcc_new_df1.set_index('date')
dtc_new_df =dtc_new_df1.set_index('date')
ease_new_df =ease_new_df1.set_index('date')

In [23]:
rcc_a = rcc_master_dataframe_cabo.tail(1)
hcc_a = hcc_master_dataframe_cabo.tail(1)
dtc_a = dtc_master_dataframe_cabo.tail(1)
ease_a = ease_master_dataframe_cabo.tail(1)

In [24]:

rcc_train  = rcc_new_df[(rcc_new_df.index.get_level_values(0) >= rcc_a['ds'].iloc[0] - timedelta(days=728) ) & (rcc_new_df.index.get_level_values(0) < rcc_a['ds'].iloc[0])]

hcc_train  = hcc_new_df[(hcc_new_df.index.get_level_values(0) >= hcc_a['ds'].iloc[0] - timedelta(days=728) ) & (hcc_new_df.index.get_level_values(0) < hcc_a['ds'].iloc[0])]

dtc_train  = dtc_new_df[(dtc_new_df.index.get_level_values(0) >= dtc_a['ds'].iloc[0] - timedelta(days=728) ) & (dtc_new_df.index.get_level_values(0) < dtc_a['ds'].iloc[0])]

ease_train = ease_new_df[(ease_new_df.index.get_level_values(0) >= ease_a['ds'].iloc[0] - timedelta(days=728) ) & (ease_new_df.index.get_level_values(0) < ease_a['ds'].iloc[0])]


In [25]:
from datetime import date
a = date(date.today().year, 1, 1)
b = date(date.today().year, 12, 31)

In [26]:
#RCC MODEL

rcc_model =rcc_train[['ds','y']]
rcc_m = Prophet(interval_width=0.95)
rcc_m.fit(rcc_model)

d0= b
d1=pd.to_datetime(d0)
d2=rcc_train['ds'].tail(1).iloc[0]
delta = d1 - d2
p= delta.days/7
P=round(p)

future = rcc_m.make_future_dataframe(periods=P, freq='W-FRI')
future

forecastr = rcc_m.predict(future)
rccforecast= forecastr[['ds', 'yhat']]

19:30:32 - cmdstanpy - INFO - Chain [1] start processing
19:30:32 - cmdstanpy - INFO - Chain [1] done processing


In [27]:
#HCC MODEL

hcc_model =hcc_train[['ds','y']]
hcc_m = Prophet(interval_width=0.95)
hcc_m.fit(hcc_model)

d0=b
d1=pd.to_datetime(d0)
d2=hcc_train['ds'].tail(1).iloc[0]
delta = d1 - d2
p= delta.days/7
P=round(p)

future = hcc_m.make_future_dataframe(periods=P, freq='W-FRI')
future

forecasth = hcc_m.predict(future)
hccforecast= forecasth[['ds', 'yhat']]

19:30:32 - cmdstanpy - INFO - Chain [1] start processing
19:30:32 - cmdstanpy - INFO - Chain [1] done processing


In [28]:
#DTC MODEL

dtc_model =dtc_train[['ds','y']]
dtc_m = Prophet(interval_width=0.95)
dtc_m.fit(dtc_model)

d0=b
d1=pd.to_datetime(d0)
d2=dtc_train['ds'].tail(1).iloc[0]
delta = d1 - d2
p= delta.days/7
P=round(p)

future = dtc_m.make_future_dataframe(periods=P, freq='W-FRI')
future

forecastd = dtc_m.predict(future)
dtcforecast= forecastd[['ds', 'yhat']]

19:30:32 - cmdstanpy - INFO - Chain [1] start processing
19:30:32 - cmdstanpy - INFO - Chain [1] done processing


In [29]:
#EASE MODEL

ease_model =ease_train[['ds','y']]
ease_m = Prophet(interval_width=0.95)
ease_m.fit(ease_model)

d0=b
d1=pd.to_datetime(d0)
d2=ease_train['ds'].tail(1).iloc[0]
delta = d1 - d2
p= delta.days/7
P=round(p)

future = ease_m.make_future_dataframe(periods=P, freq='W-FRI')
future

forecast = ease_m.predict(future)
easeforecast= forecast[['ds', 'yhat']]

19:30:33 - cmdstanpy - INFO - Chain [1] start processing
19:30:33 - cmdstanpy - INFO - Chain [1] done processing


In [30]:
rcc_forecast = rccforecast[rccforecast['ds'] >= rcc_a['ds'].iloc[0]]
hcc_forecast = hccforecast[rccforecast['ds'] >= hcc_a['ds'].iloc[0]]
dtc_forecast = dtcforecast[rccforecast['ds'] >= dtc_a['ds'].iloc[0]]
ease_forecast = easeforecast[rccforecast['ds'] >= ease_a['ds'].iloc[0]]

In [31]:
#RCC
rcc_data= rcc_new_df1[['ds', 'y']]
rcc_data_new = rcc_data.iloc[:-1]
rcc_data_new.set_index('ds')
rcc_forecast.rename(columns = {'yhat':'Prediction'}, inplace = True)
rcc_forecast.set_index('ds')

#HCC
hcc_data= hcc_new_df1[['ds', 'y']]
hcc_data_new = hcc_data.iloc[:-1]
hcc_data_new.set_index('ds')
hcc_forecast.rename(columns = {'yhat':'Prediction'}, inplace = True)
hcc_forecast.set_index('ds')

#DTC
dtc_data= dtc_new_df1[['ds', 'y']]
dtc_data_new = dtc_data.iloc[:-1]
dtc_data_new.set_index('ds')
dtc_forecast.rename(columns = {'yhat':'Prediction'}, inplace = True)
dtc_forecast.set_index('ds')

#EASE
ease_data= ease_new_df1[['ds', 'y']]
ease_data_new = ease_data.iloc[:-1]
ease_data_new.set_index('ds')
ease_forecast.rename(columns = {'yhat':'Prediction'}, inplace = True)
ease_forecast.set_index('ds')

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Prediction
ds,
2023-07-21,147.029617
2023-07-28,147.391194
2023-08-04,147.752770
2023-08-11,148.114347
2023-08-18,148.475923
2023-08-25,148.837500
2023-09-01,149.199076
2023-09-08,149.560653
2023-09-15,149.922230


In [32]:
rcc_future_forecasts= pd.concat([rcc_data_new,rcc_forecast])
hcc_future_forecasts= pd.concat([hcc_data_new,hcc_forecast])
dtc_future_forecasts= pd.concat([dtc_data_new,dtc_forecast])
ease_future_forecasts= pd.concat([ease_data_new,ease_forecast])

In [33]:
rcc_future_forecasts.set_index('ds')
hcc_future_forecasts.set_index('ds')
dtc_future_forecasts.set_index('ds')
ease_future_forecasts.set_index('ds')

,y,Prediction
ds,,
2021-01-08,90.0,NaN
2021-01-15,81.0,NaN
2021-01-22,57.0,NaN
2021-01-29,92.0,NaN
2021-02-05,89.0,NaN
...,...,...
2023-12-01,NaN,153.899571
2023-12-08,NaN,154.261148
2023-12-15,NaN,154.622724


In [34]:
rcc_future_forecasts['Forecast_totals'] = rcc_future_forecasts.fillna(0)['y'] + rcc_future_forecasts.fillna(0)['Prediction']
hcc_future_forecasts['Forecast_totals'] = hcc_future_forecasts.fillna(0)['y'] + hcc_future_forecasts.fillna(0)['Prediction']
dtc_future_forecasts['Forecast_totals'] = dtc_future_forecasts.fillna(0)['y'] + dtc_future_forecasts.fillna(0)['Prediction']
ease_future_forecasts['Forecast_totals'] = ease_future_forecasts.fillna(0)['y'] + ease_future_forecasts.fillna(0)['Prediction']


In [35]:
rcc_forecast_range=pd.date_range(start=a, periods=52, freq='W-FRI')
hcc_forecast_range=pd.date_range(start=a, periods=52, freq='W-FRI')
dtc_forecast_range=pd.date_range(start=a, periods=52, freq='W-FRI')
ease_forecast_range=pd.date_range(start=a, periods=52, freq='W-FRI')

In [36]:
s_e=rcc_train
s_e=s_e.reset_index()
s= s_e.head(1)
e= s_e.tail(1)

In [37]:
#RCC
rcc_future_forecasts.rename(columns = {'ds':'Date'}, inplace = True)
rcc_future_forecasts.rename(columns = {'y':'RCC_Outs'}, inplace = True)
rcc_future_forecasts['Date']=pd.to_datetime(rcc_future_forecasts['Date'])
rcc_future_forecasts

#HCC
hcc_future_forecasts.rename(columns = {'ds':'Date'}, inplace = True)
hcc_future_forecasts.rename(columns = {'y':'HCC_Outs'}, inplace = True)
hcc_future_forecasts['Date']=pd.to_datetime(hcc_future_forecasts['Date'])
hcc_future_forecasts

#DTC
dtc_future_forecasts.rename(columns = {'ds':'Date'}, inplace = True)
dtc_future_forecasts.rename(columns = {'y':'DTC_Outs'}, inplace = True)
dtc_future_forecasts['Date']=pd.to_datetime(dtc_future_forecasts['Date'])
dtc_future_forecasts

#EASE
ease_future_forecasts.rename(columns = {'ds':'Date'}, inplace = True)
ease_future_forecasts.rename(columns = {'y':'EASE_Outs'}, inplace = True)
ease_future_forecasts['Date']=pd.to_datetime(ease_future_forecasts['Date'])
ease_future_forecasts



,Date,EASE_Outs,Prediction,Forecast_totals
157,2021-01-08,90.0,NaN,90.000000
158,2021-01-15,81.0,NaN,81.000000
159,2021-01-22,57.0,NaN,57.000000
160,2021-01-29,92.0,NaN,92.000000
161,2021-02-05,89.0,NaN,89.000000
...,...,...,...,...
123,2023-12-01,NaN,153.899571,153.899571
124,2023-12-08,NaN,154.261148,154.261148
125,2023-12-15,NaN,154.622724,154.622724
126,2023-12-22,NaN,154.984301,154.984301


In [38]:
#RCC

rcc_df_Fut = rcc_future_forecasts.reset_index()
rcc_df_Fut['Year'] = pd.PeriodIndex(rcc_df_Fut.Date, freq='y')
rcc_df_Fut['Quarter'] = rcc_df_Fut.Date.dt.quarter
weekend= rcc_master_dataframe_cabo.tail(2)
rcc_df_Fut['Week_Ending_date'] = weekend['date'].iloc[0]
rcc_df_Fut['Training_Start_date'] = s['date'].iloc[0]
rcc_df_Fut['Training_End_date'] = e['date'].iloc[0]
rcc_df_Fut['Sales_Type'] = 'COMMERCIAL'
rcc_df_Fut['Model_Type'] = 'RCC-PROPHET'



In [39]:
#HCC

hcc_df_Fut = hcc_future_forecasts.reset_index()
hcc_df_Fut['Year'] = pd.PeriodIndex(hcc_df_Fut.Date, freq='y')
hcc_df_Fut['Quarter'] = rcc_df_Fut.Date.dt.quarter
weekend= hcc_master_dataframe_cabo.tail(2)
hcc_df_Fut['Week_Ending_date'] = weekend['date'].iloc[0]
hcc_df_Fut['Training_Start_date'] = s['date'].iloc[0]
hcc_df_Fut['Training_End_date'] = e['date'].iloc[0]
hcc_df_Fut['Sales_Type'] = 'COMMERCIAL'
hcc_df_Fut['Model_Type'] = 'HCC-PROPHET'


In [40]:
#DTC

dtc_df_Fut = dtc_future_forecasts.reset_index()
dtc_df_Fut['Year'] = pd.PeriodIndex(dtc_df_Fut.Date, freq='y')
dtc_df_Fut['Quarter'] = dtc_df_Fut.Date.dt.quarter
weekend= dtc_master_dataframe_cabo.tail(2)
dtc_df_Fut['Week_Ending_date'] = weekend['date'].iloc[0]
dtc_df_Fut['Training_Start_date'] = s['date'].iloc[0]
dtc_df_Fut['Training_End_date'] = e['date'].iloc[0]
dtc_df_Fut['Sales_Type'] = 'COMMERCIAL'
dtc_df_Fut['Model_Type'] = 'DTC-PROPHET'

In [41]:
#EASE

ease_df_Fut = ease_future_forecasts.reset_index()
ease_df_Fut['Year'] = pd.PeriodIndex(ease_df_Fut.Date, freq='y')
ease_df_Fut['Quarter'] = ease_df_Fut.Date.dt.quarter
weekend= ease_master_dataframe_cabo.tail(2)
ease_df_Fut['Week_Ending_date'] = weekend['date'].iloc[0]
ease_df_Fut['Training_Start_date'] = s['date'].iloc[0]
ease_df_Fut['Training_End_date'] = e['date'].iloc[0]
ease_df_Fut['Sales_Type'] = 'COMMERCIAL'
ease_df_Fut['Model_Type'] = 'EASE-PROPHET'

In [42]:
ease_df_Fut

,index,Date,EASE_Outs,Prediction,Forecast_totals,Year,Quarter,Week_Ending_date,Training_Start_date,Training_End_date,Sales_Type,Model_Type
0,157,2021-01-08,90.0,NaN,90.000000,2021,1,2023-07-14,2021-07-23,2023-07-14,COMMERCIAL,EASE-PROPHET
1,158,2021-01-15,81.0,NaN,81.000000,2021,1,2023-07-14,2021-07-23,2023-07-14,COMMERCIAL,EASE-PROPHET
2,159,2021-01-22,57.0,NaN,57.000000,2021,1,2023-07-14,2021-07-23,2023-07-14,COMMERCIAL,EASE-PROPHET
3,160,2021-01-29,92.0,NaN,92.000000,2021,1,2023-07-14,2021-07-23,2023-07-14,COMMERCIAL,EASE-PROPHET
4,161,2021-02-05,89.0,NaN,89.000000,2021,1,2023-07-14,2021-07-23,2023-07-14,COMMERCIAL,EASE-PROPHET
...,...,...,...,...,...,...,...,...,...,...,...,...
151,123,2023-12-01,NaN,153.899571,153.899571,2023,4,2023-07-14,2021-07-23,2023-07-14,COMMERCIAL,EASE-PROPHET
152,124,2023-12-08,NaN,154.261148,154.261148,2023,4,2023-07-14,2021-07-23,2023-07-14,COMMERCIAL,EASE-PROPHET
153,125,2023-12-15,NaN,154.622724,154.622724,2023,4,2023-07-14,2021-07-23,2023-07-14,COMMERCIAL,EASE-PROPHET
154,126,2023-12-22,NaN,154.984301,154.984301,2023,4,2023-07-14,2021-07-23,2023-07-14,COMMERCIAL,EASE-PROPHET


In [43]:

rcc_hcc_one = rcc_df_Fut.append(hcc_df_Fut)
rcc_hcc_dtc = rcc_hcc_one.append(dtc_df_Fut)
all_one = rcc_hcc_dtc.append(ease_df_Fut)


In [44]:
final_df= all_one[['Date','Year','Quarter','Week_Ending_date','Sales_Type','Model_Type','Training_Start_date','Training_End_date','RCC_Outs','HCC_Outs','DTC_Outs','EASE_Outs','Prediction','Forecast_totals']]

In [45]:
final_df.reset_index(drop=True, inplace=True)

In [46]:
s_e=final_df
s_e=s_e.reset_index()
e= s_e.Week_Ending_date.iloc[0]

In [47]:
file_name_1 = "PROPHET_MODEL.csv"
final_df.to_csv(file_name_1,index=False)

In [48]:
a = 'PROPHET_MODEL.csv'
b = str(e)

In [49]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name_1, 'forecastingpoc', 'inputfiles/'+b+' - '+a)